In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
index_to_save = 8

<IPython.core.display.Javascript object>

In [4]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [5]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_iv",
    "Features": "Chemical + Mineralogical + Properties CS Less",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [6]:
df = pd.read_csv("../../../../../data/processed/partner_iv/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [7]:
df_copy = df.copy()
df_copy = df.drop(
    [
        # Properties
        "CS2",  # 2-day Compressive Strength
        # Removing One-Hot encoding variables
        "cem_type_CEM B",
        "cem_type_CEM C",
    ],
    axis=1,
).copy()

<IPython.core.display.Javascript object>

## Feature Engineering

In [8]:
# Feature Engineering over Chemical Features
ch_features = ["CaO", "Ca(OH)2", "Periclasio (MgO)", "K2SO4", "Cl-", "SO3"]
df_copy["std_ch_feats"] = df_copy[ch_features].std(ddof=0, axis=1)

# df_copy["ratio_MgO_to_CaO"] = df_copy["Periclasio (MgO)"] / df_copy["CaO"]

# Feature Engineering over Mineralogical Features
mi_features_set1 = [
    "Alite_M1 C3S M1",
    "Alite_M3 C3S M3",
    "Anidrite",
    "Aphthitalite – (K,Na)3(SO4)2",
    "Belite_beta",
    "C3A cub",
    "C3A tot",
    "C3A_ortho",
    "C3S_CS (taglia dei cristalliti C3S)",
    "C4AF",
    "CO2_XRD",
    "Calce libera",
    "Calcite – CaCO3",
    "Emiidrato",
    "Gesso",
    "Langbeinite – MgK2(SO4)2",
    "Quartz",
    "SO3_XRD",
]

df_copy["std_mi_set1_feats"] = df_copy[mi_features_set1].std(ddof=0, axis=1)

<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [9]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [10]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.628 (0.023)
MAE: -2.042 (0.023)
MAPE: -0.042 (0.000)
R2: 0.842 (0.003)


******
[TEST]
******
RMSE: -2.771 (0.098)
MAE: -2.156 (0.073)
MAPE: -0.045 (0.002)
R2: 0.823 (0.014)




<IPython.core.display.Javascript object>

In [11]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   

  Cross Validation RMSE Test            MAE Test           MAPE Test  \
                        mean       std      mean       std      mean   
0   Repeated KFold  2.771253  0.097987  2.155827  0.073311  0.044607   

              R2 Test            
        std      mean       std  
0  0.001802  0.823136  0.014117

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [12]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.993 (0.146)
MAE: -1.542 (0.128)
MAPE: -0.032 (0.003)
R2: 0.906 (0.015)


******
[TEST]
******
RMSE: -3.628 (1.082)
MAE: -2.823 (0.869)
MAPE: -0.059 (0.018)
R2: 0.606 (0.279)




<IPython.core.display.Javascript object>

In [13]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.628234  1.081564  2.823076  0.869470   
1              Repeated KFold  2.771253  0.097987  2.155827  0.073311   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.058569  0.017751  0.605904  0.278971  
1  0.044607  0.001802  0.823136  0.014117

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [14]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.472 (0.145)
MAE: -1.934 (0.086)
MAPE: -0.040 (0.002)
R2: 0.857 (0.017)


******
[TEST]
******
RMSE: -4.244 (1.766)
MAE: -3.420 (1.426)
MAPE: -0.071 (0.030)
R2: 0.468 (0.534)




<IPython.core.display.Javascript object>

In [15]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   
2  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.628234  1.081564  2.823076  0.869470   
1              Repeated KFold  2.771253  0.097987  2.155827  0.073311   
2           Time Series Split  4.243554  1.766497  3.419989  1.426129   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.058569  0.017751  0.605904  0.278971  
1  0.044607  0.001802  0.823136  0.014117  
2  0.070788  0.030430  0.468258  0.533894

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [16]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 2.571 (0.000)
MAE: 1.997 (0.000)
MAPE: 0.041 (0.000)
R2: 0.852 (0.000)


******
[TEST]
******
RMSE: 3.225 (0.000)
MAE: 2.485 (0.000)
MAPE: 0.052 (0.000)
R2: 0.734 (0.000)




<IPython.core.display.Javascript object>

In [17]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   
2  Chemical + Mineralogical + Properties CS Less  Linear Regression   
3  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.628234  1.081564  2.823076  0.869470   
1           Out of time Split  3.224607  0.000000  2.485490  0.000000   
2              Repeated KFold  2.771253  0.097987  2.155827  0.073311   
3           Time Series Split  4.243554  1.766497  3.419989  1.426129   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.058569  0.017751  0.605904  0.278971  
1  0.051980  0.000000  0.734202  0.000000  
2  0.044607  0.001802  0.823136  0.014117  
3  0.070788  0.030430  0.468258  0.533894

<IPython.core.display.Javascript object>

In [18]:
pipeline.named_steps["estimator"].coef_

array([  0.91015035,  -1.146396  ,  -0.87761477,   0.17915474,
         0.1117529 ,   0.25641583,   0.67042262,  -1.65639039,
         0.16693924, -16.78956198,  -8.91143946,  72.25079262,
        -3.01425615,  11.55093538,  16.38518375,  -2.89459279,
        -4.82065521,  11.46881241,  11.54608779,   4.09903835,
         4.30626053,   1.18199036,   0.84400644,   3.62348813,
         1.03960003,   0.83607077,   0.81266182,   3.88707102,
         2.33673792,   3.04994163,  63.66294909,   2.47131348,
        -3.91213856,  -1.24819314, -11.63175115])

<IPython.core.display.Javascript object>

In [19]:
pipeline.named_steps["estimator"].intercept_

49.225848563968704

<IPython.core.display.Javascript object>

In [20]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [21]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['SO3', 'Cl-', 'Blaine', 'ph2oimm', 'Initial Setting Time', 'Soundness',
       'flow', 'residuo 24 micron', 'R_wp', 'Alite_M3 C3S M3',
       'Alite_M1 C3S M1', 'Alite_Sum C3S tot', 'Ratio_M1 (rapporto M1/M3)',
       'C3S_CS (taglia dei cristalliti C3S)', 'Belite_beta', 'C3A cub',
       'C3A_ortho', 'C3A tot', 'C4AF', 'CaO', 'Ca(OH)2', 'Calce libera',
       'Periclasio (MgO)', 'Quartz', 'K2SO4', 'Langbeinite – MgK2(SO4)2',
       'Aphthitalite – (K,Na)3(SO4)2', 'Gesso', 'Emiidrato', 'Anidrite',
       'Calcite – CaCO3', 'SO3_XRD', 'CO2_XRD', 'std_ch_feats',
       'std_mi_set1_feats'],
      dtype='object')

<IPython.core.display.Javascript object>

In [22]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(35,)

<IPython.core.display.Javascript object>

In [23]:
x.columns.shape

(35,)

<IPython.core.display.Javascript object>

In [24]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [25]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
Alite_Sum C3S tot,72.250793
Calcite – CaCO3,63.662949
Belite_beta,16.385184
C3S_CS (taglia dei cristalliti C3S),11.550935
C4AF,11.546088
C3A tot,11.468812
Ca(OH)2,4.306261
CaO,4.099038
Gesso,3.887071
Quartz,3.623488


<IPython.core.display.Javascript object>

In [26]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   
2  Chemical + Mineralogical + Properties CS Less  Linear Regression   
3  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  3.628234  1.081564   
1                                {"Test Size": 0.2}  3.224607  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  2.771253  0.097987   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  4.243554  1.766497   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  2.823076  0.869470  0.058569  0.017751  0.605904  0.278971  
1  2.485490  0.000000  0.051980  0.000000  0.734202  0.000000  
2  2.155827  0.073311  0.044607  0.001802  0.823136  0.014117  
3  3.419989  1.426129  0.070788  0.030430  0.468258  0.533894

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [27]:
path = "../../../../../reports/results/local_models/partner_iv/all_cements/full/"
filename = f"linear_regression_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [28]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_iv/all_cements/grouped/"
filename = f"linear_regression_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [29]:
df_results_to_save

Category Company Data Shape Timesteps  \
                                              
0  Local Model   partner_iv  (958, 35)       NaN   
1  Local Model   partner_iv  (958, 35)       NaN   
2  Local Model   partner_iv  (958, 35)       NaN   
3  Local Model   partner_iv  (958, 35)       NaN   

                                        Features              Model  \
                                                                      
0  Chemical + Mineralogical + Properties CS Less  Linear Regression   
1  Chemical + Mineralogical + Properties CS Less  Linear Regression   
2  Chemical + Mineralogical + Properties CS Less  Linear Regression   
3  Chemical + Mineralogical + Properties CS Less  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   1.992803  0.145989  ...   
1                                {"Test Size": 0.2}   2.571231  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   2.627990  0.023050  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   2.472222  0.145392  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.906427  0.014892  3.628234  1.081564  2.823076  0.869470  0.058569   
1  0.852239  0.000000  3.224607  0.000000  2.485490  0.000000  0.051980   
2  0.842061  0.003434  2.771253  0.097987  2.155827  0.073311  0.044607   
3  0.856905  0.016897  4.243554  1.766497  3.419989  1.426129  0.070788   

              R2 Test            
        std      mean       std  
0  0.017751  0.605904  0.278971  
1  0.000000  0.734202  0.000000  
2  0.001802  0.823136  0.014117  
3  0.030430  0.468258  0.533894  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>